In [1]:
#Importar librerias 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium

In [2]:
#Leer dataframe original
df1 = pd.read_csv("data/Operacion_Pasajeros_despacho_vehiculos_carretera_Dataframe_06072021.csv")

In [3]:
#Limpieza y adecuación de datos
df1["FECHA_DESPACHO"]=df1["FECHA_DESPACHO"].apply(lambda x:x[:10])

In [6]:
df1["FECHA_DESPACHO"]=pd.to_datetime(df1["FECHA_DESPACHO"])

In [7]:
date=pd.Timestamp('2021-07-06 00:00:00')
indexborrar=df1[df1["FECHA_DESPACHO"]>date].index
df1.drop(index=indexborrar,inplace=True)

In [8]:
#Crear columna Ruta concatenando los códigos de los municipios origen con municipios destino
df1["Ruta"]=df1["MUNICIPIO_ORIGEN_RUTA"].astype(str)+"-"+df1["MUNICIPIO_DESTINO_RUTA"].astype(str)

In [9]:
#Cargar dataframe auxiliar para nombres
df2=pd.read_csv("data/Departamentos_y_municipios_de_Colombia.csv")

In [10]:
#Cargar dataframe auxiliar para coordenadas
dfcoord=pd.read_csv("data/Divipola_coords.csv", sep=';')

In [11]:
#Crear nuevo dataframe para analizar rutas sumando pasajeros
pasajerosruta=df1.groupby("Ruta").sum()[["PASAJEROS"]].sort_values(by="PASAJEROS",ascending=False)
pasajerosruta.reset_index(inplace=True)

In [28]:
pasajerosruta2

,Ruta,PASAJEROS,COD_ORIGEN,COD_DESTINO,NOM_ORIGEN,NOM_DESTINO,LON_ORIGEN,LAT_ORIGEN,LON_DESTINO,LAT_DESTINO,cum_percent,TOP
0,5001-5615,1511880,5001,5615,Medellín,Rionegro,-75.581713,6.246716,-75.376769,6.147224,1.215338,True
1,11001-76001,1429712,11001,76001,Bogotá D.C.,Cali,-74.107359,4.650173,-76.521565,3.414414,2.364624,True
2,76001-76520,1425966,76001,76520,Cali,Palmira,-76.521565,3.414414,-76.298583,3.532349,3.510898,True
3,63001-63401,1307838,63001,63401,Armenia,La Tebaida,-75.680751,4.535990,-75.786460,4.453650,4.562215,True
4,11001-25307,1187654,11001,25307,Bogotá D.C.,Girardot,-74.107359,4.650173,-74.797795,4.313078,5.516921,True
...,...,...,...,...,...,...,...,...,...,...,...,...
367,86885-86568,79115,86885,86568,Villagarzón,Puerto Asís,-76.617162,1.028941,-76.497273,0.505047,79.743636,True
368,11001-15047,78871,11001,15047,Bogotá D.C.,Aquitania,-74.107359,4.650173,-72.883985,5.518604,79.807037,True
369,8001-70001,78584,8001,70001,Barranquilla,Sincelejo,-74.795259,10.975927,-75.395484,9.302333,79.870208,True
370,52001-52110,78044,52001,52110,Pasto,Buesaco,-77.278576,1.212124,-77.156480,1.381434,79.932944,True


In [15]:
#Separar Rutas 
pasajerosruta[['COD_ORIGEN', 'COD_DESTINO']] = pasajerosruta['Ruta'].str.split('-', 1, expand=True).astype(int)

In [20]:
#Nombrar origen y destino
df2["CÓDIGO DANE DEL MUNICIPIO"]=df2["CÓDIGO DANE DEL MUNICIPIO"].str.replace(",","")
df2["CÓDIGO DANE DEL MUNICIPIO"] = df2["CÓDIGO DANE DEL MUNICIPIO"].astype(int)
pasajerosruta["NOM_ORIGEN"]=pasajerosruta["COD_ORIGEN"].map(df2.set_index('CÓDIGO DANE DEL MUNICIPIO')['MUNICIPIO'])
pasajerosruta["NOM_DESTINO"]=pasajerosruta["COD_DESTINO"].map(df2.set_index('CÓDIGO DANE DEL MUNICIPIO')['MUNICIPIO'])

In [18]:
#Determinar coordenadas origen y destino de las rutas
pasajerosruta["LON_ORIGEN"]=pasajerosruta["COD_ORIGEN"].map(dfcoord.set_index('Código Municipio')['Longitud'])
pasajerosruta["LAT_ORIGEN"]=pasajerosruta["COD_ORIGEN"].map(dfcoord.set_index('Código Municipio')['Latitud'])
pasajerosruta["LON_DESTINO"]=pasajerosruta["COD_DESTINO"].map(dfcoord.set_index('Código Municipio')['Longitud'])
pasajerosruta["LAT_DESTINO"]=pasajerosruta["COD_DESTINO"].map(dfcoord.set_index('Código Municipio')['Latitud'])

In [23]:
#Determinar las rutas top en movimiento de pasajeros para empezar visualizaciones
pasajerosruta["cum_percent"] = 100 * (pasajerosruta["PASAJEROS"].cumsum()/pasajerosruta["PASAJEROS"].sum()) 

In [25]:
#Treshold en 80%
rows_top=pasajerosruta[pasajerosruta["cum_percent"]<80].index
rows_notop=pasajerosruta[pasajerosruta["cum_percent"]>80].index
pasajerosruta.loc[rows_top,"TOP"]=True
pasajerosruta.loc[rows_notop,"TOP"]=False

In [27]:
pasajerosruta2=pasajerosruta.copy()
pasajerosruta2.drop(pasajerosruta2[pasajerosruta2["TOP"] ==False].index, inplace = True)

In [43]:
#GRAFICAR LAS RUTAS TOP QUE TRANSPORTAN PASAJEROS
centroid_lat=4.624335
centroid_lon=-74.063644

fff = folium.Map([centroid_lat, centroid_lon], zoom_start=5)
for _, row in pasajerosruta2.iterrows():
    folium.PolyLine([[row['LAT_ORIGEN'], row['LON_ORIGEN']], 
                     [row['LAT_DESTINO'], row['LON_DESTINO']]]).add_to(fff)
fff

In [33]:
#Ahora vamos a generar las rutas que tienen más despachos
despachosruta=df1.groupby("Ruta").sum()[["DESPACHOS"]].sort_values(by="DESPACHOS",ascending=False)
despachosruta.reset_index(inplace=True)
despachosruta

,Ruta,DESPACHOS
0,11001-25740,365436
1,66001-66682,293842
2,5045-5837,216249
3,11001-25307,212533
4,11001-15759,186152
...,...,...
2841,15001-13001,1
2842,11001-19693,1
2843,13430-44430,1
2844,13001-13760,1


In [34]:
#Llamar las otras columnas para que quede similar
despachosruta[['COD_ORIGEN', 'COD_DESTINO']] = despachosruta['Ruta'].str.split('-', 1, expand=True).astype(int)
despachosruta["NOM_ORIGEN"]=despachosruta["COD_ORIGEN"].map(df2.set_index('CÓDIGO DANE DEL MUNICIPIO')['MUNICIPIO'])
despachosruta["NOM_DESTINO"]=despachosruta["COD_DESTINO"].map(df2.set_index('CÓDIGO DANE DEL MUNICIPIO')['MUNICIPIO'])
despachosruta["LON_ORIGEN"]=despachosruta["COD_ORIGEN"].map(dfcoord.set_index('Código Municipio')['Longitud'])
despachosruta["LAT_ORIGEN"]=despachosruta["COD_ORIGEN"].map(dfcoord.set_index('Código Municipio')['Latitud'])
despachosruta["LON_DESTINO"]=despachosruta["COD_DESTINO"].map(dfcoord.set_index('Código Municipio')['Longitud'])
despachosruta["LAT_DESTINO"]=despachosruta["COD_DESTINO"].map(dfcoord.set_index('Código Municipio')['Latitud'])

In [37]:
despachosruta["cum_percent"] = 100 * (despachosruta["DESPACHOS"].cumsum()/despachosruta["DESPACHOS"].sum()) 
#Treshold en 80%
rows_top2=despachosruta[despachosruta["cum_percent"]<80].index
rows_notop2=despachosruta[despachosruta["cum_percent"]>80].index
despachosruta.loc[rows_top2,"TOP"]=True
despachosruta.loc[rows_notop2,"TOP"]=False

In [39]:
despachosruta2=despachosruta.copy()
despachosruta2.drop(despachosruta2[despachosruta2["TOP"] ==False].index, inplace = True)

In [40]:
despachosruta2

,Ruta,DESPACHOS,COD_ORIGEN,COD_DESTINO,NOM_ORIGEN,NOM_DESTINO,LON_ORIGEN,LAT_ORIGEN,LON_DESTINO,LAT_DESTINO,cum_percent,TOP
0,11001-25740,365436,11001,25740,Bogotá D.C.,Sibaté,-74.107359,4.650173,-74.258992,4.489764,2.059916,True
1,66001-66682,293842,66001,66682,Pereira,Santa Rosa de Cabal,-75.717123,4.805680,-75.620011,4.867526,3.716266,True
2,5045-5837,216249,5045,5837,Apartadó,Turbo,-76.625961,7.883417,-76.729158,8.089116,4.935235,True
3,11001-25307,212533,11001,25307,Bogotá D.C.,Girardot,-74.107359,4.650173,-74.797795,4.313078,6.133256,True
4,11001-15759,186152,11001,15759,Bogotá D.C.,Sogamoso,-74.107359,4.650173,-72.923048,5.725689,7.182571,True
...,...,...,...,...,...,...,...,...,...,...,...,...
400,17001-17777,10690,17001,17777,Manizales,Supía,-75.491049,5.057688,-75.649660,5.446843,79.723542,True
401,18001-18205,10687,18001,18205,Florencia,Curillo,-75.608560,1.617923,-75.919204,1.033474,79.783783,True
402,11001-52356,10679,11001,52356,Bogotá D.C.,Ipiales,-74.107359,4.650173,-77.646401,0.827734,79.843979,True
403,11001-15572,10603,11001,15572,Bogotá D.C.,Puerto Boyacá,-74.107359,4.650173,-74.587999,5.977935,79.903747,True


In [42]:
#GRAFICAR LAS RUTAS TOP QUE MÁS REALIZAN DESPACHOS
centroid_lat=4.624335
centroid_lon=-74.063644

ff = folium.Map([centroid_lat, centroid_lon], zoom_start=5)
for _, row in despachosruta2.iterrows():
    folium.PolyLine([[row['LAT_ORIGEN'], row['LON_ORIGEN']], 
                     [row['LAT_DESTINO'], row['LON_DESTINO']]]).add_to(ff)
ff

## Last we realized that the right step is to utilize dash to create powerful graphics, we will work towards that